# Introduction
In this tutorial, we will learn how to use the PuLP library in Python to solve linear programming problems. Linear programming is a method to achieve the best outcome in a mathematical model whose requirements are represented by linear constraints.

In [1]:
#Install pulp
!pip install pulp

## Problem Formulation
Let's consider a simple problem: 

Maximize: $$Z = 4x + 3y$$ 
Subject to: 
$$2x + y \leq 20$$ 
$$x + y \leq 12$$ 
$$x, y \geq 0$$

In [2]:
# Import the library
from pulp import *

# Create a problem variable:
prob = LpProblem("Maximize the profit Z", LpMaximize)

# Create problem variables:
x = LpVariable("x", lowBound=0, upBound=None) # x>=0
y = LpVariable("y", lowBound=0, upBound=None) # y>=0


d:\miniforge3\envs\base2\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In linear programming problems, the objective function represents the quantity which needs to be minimized or maximized. It does not have constraints like `<=` or `>=`. On the other hand, constraints are the restrictions or limitations on the variables. They have a certain form based on the problem requirements, often represented with `<=`, `>=`, or `==`.

In [3]:

# The objective function and constraints are added using the += operator to our model.
# Objective function Z
prob += 4*x + 3*y, "Profit" 

# Constraints
prob += 2*x + y <= 20
prob += x + y <= 12

<b>Note: The names of variables or constraints must be unique and special characters must not appear, e.g. `=`,`<`,`>`. 

In [4]:
# Problem
prob

Maximize_the_profit_Z:
MAXIMIZE
4*x + 3*y + 0
SUBJECT TO
_C1: 2 x + y <= 20

_C2: x + y <= 12

VARIABLES
x Continuous
y Continuous

In [5]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal production amount of x and y
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Print the optimal profit
print("Total profit is: ", value(prob.objective))

Status: Optimal
x = 8.0
y = 4.0
Total profit is:  44.0


# Second Example: Solving a Knapsack Problem
In this example, we will solve a knapsack problem. We have a set of items, each with a weight and a value, and we want to determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

Maximize: 
$$Z = 50x_1 + 100x_2 + 120x_3$$ 
Subject to: 
$$10x_1 + 20x_2 + 30x_3 \leq 50$$ 
$$x_1, x_2, x_3  \in \{0,1\}$$

In [6]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem(name="Knapsack Problem", sense=LpMaximize)

# The 3 binary variables that can only take values of 0 or 1
x1 = LpVariable(name="Item1", cat='Binary')
x2 = LpVariable(name="Item2", cat='Binary')
x3 = LpVariable(name="Item3", cat='Binary')

# The objective function is added to 'prob'
prob += lpSum([50*x1, 100*x2, 120*x3]), "Total Value of Items in Knapsack"

# Constraint
prob += lpSum([10*x1, 20*x2, 30*x3]) <= 50, "Total Weight of Items in Knapsack"

In [7]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal solution
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Print the optimal total value
print("Total value of items in knapsack is: ", value(prob.objective))

Status: Optimal
Item1 = 0
Item2 = 1
Item3 = 1
Total value of items in knapsack is:  220


# Third Example: Using Binary Variables as Switches
In this example, we will use a binary variable as a switch to control whether certain constraints are active or not. This is a common technique in linear programming when we want to model conditional constraints.

Maximize: $$Z = b_1 + b_2 + b_3$$ 
Subject to: 
$$x + y + M(1-b_1) \geq 50$$ 
$$x + 2y + M b_1 \leq 100$$ 
$$3x + 2y + M(1-b_2) \geq 50$$ 
$$-x + 5y + M b_3 > 75$$ 
$$x \geq 0, x \leq 8$$
$$y \geq 0$$
$$b_1, b_2, b_3 \in \{0,1\}$$

In [8]:
prob = LpProblem(name="Switch Problem", sense=LpMaximize)

# The variables are created
x = LpVariable(name="x", lowBound=0,upBound=8)
y = LpVariable(name="y", lowBound=0)
b1 = LpVariable(name="b1", cat='Binary')
b2 = LpVariable(name="b2", cat='Binary')
b3 = LpVariable(name="b3", cat='Binary')

# The objective function is added to 'prob' first
prob += lpSum([b1,b2,b3]), "Total Value"


M = 1000  # A very large number
eps = 0.00001# A very small number
prob += lpSum([x, y]) + M*(1-b1)>= 50 , "Weight constraint when b1 is 1"
prob += lpSum([x, 2*y]) + M*b1 <= 100 , "Weight constraint when b1 is 0"
prob += lpSum([3*x, 2*y]) + M*(1-b2)>= 50 , "Weight constraint when b2 is 1"
# It is not possible to model sharp inequalities `>` or `<` in solver, 
# in order to model them a small epsilon value is added artificially to the non-sharp equations.
prob += lpSum([-x, 5*y]) + M*b3 >= 75+eps , "Weight constraint when b3 is 0"


In [9]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal solution
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Print the optimal total value
print("Total value is: ", value(prob.objective))

Status: Optimal
b1 = 0
b2 = 1
b3 = 1
x = 8.0
y = 13.0
Total value is:  2
